In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly

import test
import mylayers
import util

# Used to save temporary variables
import pickle


## Read data

In [6]:
df_train = pd.read_csv("./data/fashion-mnist_train.csv")
df_valid = pd.read_csv("./data/fashion-mnist_test.csv")


In [7]:
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_train.shape

(60000, 785)

### Shuffle the data set

In [9]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)


### Seperate feature column and target columns

In [11]:
X_train = np.array(df_train.drop('label', axis=1), dtype="f")
Y_train = df_train.loc[:, 'label']
X_valid = np.array(df_valid.drop('label', axis=1), dtype="f")
Y_valid = df_valid.loc[:, 'label']


### One hot encoding Y

In [12]:
encoder = util.onehot_encoder(Y_train)
Y_train = encoder.encoding(Y_train)
Y_valid = encoder.encoding(Y_valid)


In [14]:
X_train = X_train[:100]
Y_train = Y_train[:100]

## Build layers

In [15]:
layers = []
# ---Input Layer---
layers.append(mylayers.InputLayer(X_train))
# ---Hidden Layer---
# initialize FCL with xaiver
fc_layer = mylayers.FullyConnectedLayer(X_train.shape[1], 10)
weights, biases = util.xaiver_init(X_train.shape[1], 10)
fc_layer.setWeights(weights)
fc_layer.setBiases(biases)
layers.append(fc_layer)
layers.append(mylayers.SoftmaxLayer())
# ---Output Layer---
# fc_layer = mylayers.FullyConnectedLayer(20, 10)
# weights, biases = util.xaiver_init(20, 10)
# fc_layer.setWeights(weights)
# fc_layer.setBiases(biases)
# layers.append(fc_layer)
# layers.append(mylayers.SoftmaxLayer())
# ---Objective---
layers.append(mylayers.CrossEntropy())


### layers test

In [16]:
hyperparams = {"epoch": 500, "eta": 10e-8, "batch_size": 5, "rho1": 0.9, "rho2": 0.999}


In [17]:
%time
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,ls_accuracy_train,ls_scores_valid,ls_accuracy_valid = test.adam_learn_with_validation(X_train,Y_train,X_valid,Y_valid,layers,mylayers.CrossEntropy,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 500/500 [00:19<00:00, 26.05it/s]


In [18]:
train_results = {
    "ls_scores_train": ls_scores_train,
    "Y_hat": Y_hat,
    "ls_accuracy_train": ls_accuracy_train,
    "ls_scores_valid": ls_scores_valid,
    "ls_accuracy_valid": ls_accuracy_valid,
}


In [19]:
# save result to pickle
with open("models/ANN_train_results_softmax.pkl", "wb") as f:
    pickle.dump(train_results, f)


In [20]:
# read result from pickle
with open("models/ANN_train_results_softmax.pkl", "rb") as f:
    train_results = pickle.load(f)
ls_scores_train = train_results["ls_scores_train"]
Y_hat = train_results["Y_hat"]
ls_accuracy_train = train_results["ls_accuracy_train"]
ls_scores_valid = train_results["ls_scores_valid"]
ls_accuracy_valid = train_results["ls_accuracy_valid"]


In [21]:
# graph of scores
hidden_layers = "softmax"
fig = util.draw_scores_with_validation(
    ls_scores_train, ls_scores_valid, hidden_layers, hyperparams
)
fig.show()


In [22]:
# graph of accuracys
fig = util.draw_accuracys_with_validation(
    ls_accuracy_train, ls_accuracy_valid, hidden_layers, hyperparams
)
fig.show()


### validate accuracy

In [23]:
labeled_Y_hat = encoder.decoding(Y_hat)
labeled_Y_hat


[1,
 4,
 4,
 9,
 7,
 0,
 6,
 0,
 9,
 8,
 6,
 0,
 9,
 0,
 6,
 0,
 7,
 2,
 5,
 6,
 8,
 7,
 3,
 0,
 6,
 9,
 8,
 1,
 0,
 9,
 1,
 8,
 8,
 9,
 0,
 0,
 5,
 4,
 8,
 9,
 2,
 1,
 3,
 2,
 0,
 9,
 0,
 6,
 2,
 4,
 1,
 1,
 5,
 9,
 1,
 4,
 1,
 0,
 9,
 7,
 0,
 7,
 6,
 0,
 9,
 1,
 1,
 5,
 6,
 0,
 9,
 1,
 1,
 1,
 8,
 6,
 0,
 9,
 6,
 0,
 0,
 0,
 7,
 1,
 9,
 7,
 0,
 5,
 0,
 0,
 2,
 6,
 2,
 6,
 8,
 9,
 2,
 5,
 7,
 4]

In [24]:
labeled_Y = encoder.decoding(Y_train)
labeled_Y


[7,
 8,
 8,
 5,
 9,
 1,
 2,
 6,
 6,
 2,
 5,
 0,
 7,
 1,
 6,
 0,
 6,
 2,
 9,
 1,
 2,
 4,
 8,
 0,
 4,
 9,
 1,
 0,
 0,
 5,
 1,
 6,
 2,
 7,
 3,
 4,
 9,
 9,
 4,
 8,
 2,
 6,
 0,
 2,
 1,
 5,
 4,
 4,
 6,
 9,
 1,
 3,
 2,
 5,
 8,
 9,
 5,
 1,
 1,
 2,
 3,
 8,
 3,
 1,
 2,
 8,
 0,
 9,
 4,
 0,
 5,
 6,
 5,
 2,
 6,
 2,
 0,
 5,
 2,
 1,
 1,
 6,
 9,
 7,
 5,
 2,
 9,
 2,
 6,
 3,
 2,
 4,
 4,
 2,
 9,
 7,
 5,
 8,
 7,
 0]

In [25]:
c = 0
for i in range(len(labeled_Y)):
    if labeled_Y[i] == labeled_Y_hat[i]:
        c += 1
print("Final Accuracy:", (c / len(labeled_Y)) * 100, "%")


Final Accuracy: 15.0 %
